---
## CSE 482 - Tweet Loading and Processing
### Jack Nugent
Load Tweets related to Elon Musk and prepare the text for sentiment analysis.

---

In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
import json
import pandas as pd
import re
import numpy as np

In [2]:
#!pip install emoji
import emoji

In [3]:
# Replace with Your Keys
C_KEY = None
C_SECRET = None
A_TOKEN_KEY = None
A_TOKEN_SECRET = None
BEARER = None

In [4]:
# Jack Nugent Keys
C_KEY = 'qDLzDR1kgKZrwc0nfCiknCNak'
C_SECRET = '0siNqGf4CSQVM2EL2JTYKq0fqBN7VXtTXv2MSyUAiiX5V1DBeG'
A_TOKEN_KEY = '762757536827121664-YNdvEO2FsulfLcjUJcezb5OMRx3Nlgs'
A_TOKEN_SECRET = '88DYUdSwCfWmNlBWbY9It8uFqajxE7NMMRS4moZbdjz4k'
BEARER = "AAAAAAAAAAAAAAAAAAAAAOCnhQEAAAAABJxPmDWLGJXfFdBa9Nprs%2BfFHus%3De6Qt9gjlzObpDy4HIWJWMQUXXLE6nkNUF3Drkv6QLkZezBn4JU"

---
### Loading Tweets

In [5]:
data_json = []

In [6]:
files = ['output_geotag_day1.json', 'output_geotag_day2.json']

for file in files:
    f = open(file)

    for line in f:
        try:
            data_json.append(json.loads(line))
        # Discard Tweets with invalid chars
        except:
            pass

    f.close()

In [7]:
df = pd.DataFrame(columns=['Text', 'Place_id', 'Tweet_id', 'Date', 'Time'])

for tweet in data_json:
    date, time = tweet['data']['created_at'].split('T')
    # Handles Retweets that somehow slip through filter
    if 'RT @' not in tweet['data']['text']:
        df.loc[len(df.index)] = [tweet['data']['text'], tweet['data']['geo']['place_id'], tweet['data']['id'], date, time[:-5]]

print("Number of Tweets:",len(df))
df

Number of Tweets: 695


,Text,Place_id,Tweet_id,Date,Time
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,6079430106fe2a39,1595482958764425218,2022-11-23,18:22:34
1,Twitter and Elon has the same vibes when Trump...,94965b2c45386f87,1595483029048369152,2022-11-23,18:22:51
2,@RonFilipkowski Then why do I see him so frequ...,2d83c71ce16cd187,1595483099735154688,2022-11-23,18:23:08
3,@timothy_wall @elonmusk This is hysterical. Th...,018244f41aa15ac3,1595483141879353344,2022-11-23,18:23:18
4,Elon Musk can show us the way but it’s we who ...,011add077f4d2da3,1595483162289016832,2022-11-23,18:23:23
...,...,...,...,...,...
690,@jljacobson Her opinion of him has changed dra...,01a9a39529b27f36,1595853104565686273,2022-11-24,18:53:24
691,@RonFilipkowski @elonmusk Another right wing i...,7142eb97ae21e839,1595853147792187397,2022-11-24,18:53:34
692,@MattWallace888 Another mindless shit post…it’...,9dafd05b1158873b,1595853341531471872,2022-11-24,18:54:20
693,@jbblawmd Well I burned the skin on the oven w...,dd9c503d6c35364b,1595853364713213957,2022-11-24,18:54:26


---
### Processing Text


#### Cleaning Text

In [8]:
%%time
df['Words'] = df['Text'].str.split()
df['Hashtags'] = None
for x in range(len(df)):
    # Handle '@' Mentions
    df['Words'][x] = [i for i in df['Words'][x] if not re.compile('@').match(i)]
    
    # Handle links
    df['Words'][x] = [i for i in df['Words'][x] if not re.compile('http').match(i)]
    
    # Hashtags
    df['Hashtags'][x] = [word for word in df['Words'][x] if '#' in word]
    df['Words'][x] = [word for word in df['Words'][x] if word not in df['Hashtags'][x]]
    
    # Handle Emojis and Punctuation
    words = []
    for word in df.iloc[x]['Words']:
        for c in word:
            if c in emoji.EMOJI_DATA or c in '''!()-[]{};:'"\,<>./?@#$%^&*_~''':
                word = word.replace(c, '')
        # Lowercase Words
        words.append(word.lower())
    df['Words'][x] = words
    
    # Remove Empty Values
    for word in df['Words'][x]:
        if word == '':
            df['Words'][x].remove(word)
            

CPU times: user 695 ms, sys: 14.2 ms, total: 709 ms
Wall time: 698 ms


---
### NLTK Cleaning

In [9]:
import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package wordnet to /home/nugentj3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nugentj3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nugentj3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nugentj3/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


---
#### Stopword Removal

In [10]:
stop_words = set(stopwords.words('english'))

In [11]:
# Stopword Removal
for x in range(len(df)):
    df['Words'][x] = [word for word in df['Words'][x] if word not in stop_words]
#df

---
#### Stemming

In [12]:
df['Stemmed'] = None
for x in range(len(df)):
    df['Stemmed'][x] = [SnowballStemmer(language='english').stem(word) for word in df['Words'][x]]
#df

---
#### Lemmatization

In [13]:
df['Lemmed'] = None
for x in range(len(df)):
    df['Lemmed'][x] = [WordNetLemmatizer().lemmatize(word) for word in df['Words'][x]]
#df

---
#### Part of Speech Tagging

In [14]:
#POS Tagging
df['POS_Tags'] = None
for x in range(len(df)):
    df['POS_Tags'][x] = nltk.pos_tag(df['Lemmed'][x])
df

,Text,Place_id,Tweet_id,Date,Time,Words,Hashtags,Stemmed,Lemmed,POS_Tags
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,6079430106fe2a39,1595482958764425218,2022-11-23,18:22:34,"[econusyougovamerica, approve, disapprove, elo...",[],"[econusyougovamerica, approv, disapprov, elon,...","[econusyougovamerica, approve, disapprove, elo...","[(econusyougovamerica, RB), (approve, VB), (di..."
1,Twitter and Elon has the same vibes when Trump...,94965b2c45386f87,1595483029048369152,2022-11-23,18:22:51,"[twitter, elon, vibes, trump, got, elected, la...",[],"[twitter, elon, vibe, trump, got, elect, laugh...","[twitter, elon, vibe, trump, got, elected, lau...","[(twitter, NN), (elon, NN), (vibe, NN), (trump..."
2,@RonFilipkowski Then why do I see him so frequ...,2d83c71ce16cd187,1595483099735154688,2022-11-23,18:23:08,"[see, frequently, twitter, stop, posting]",[],"[see, frequent, twitter, stop, post]","[see, frequently, twitter, stop, posting]","[(see, VB), (frequently, RB), (twitter, JJR), ..."
3,@timothy_wall @elonmusk This is hysterical. Th...,018244f41aa15ac3,1595483141879353344,2022-11-23,18:23:18,"[hysterical, thanks, laugh, prep, thanksgiving...",[],"[hyster, thank, laugh, prep, thanksgiv, need]","[hysterical, thanks, laugh, prep, thanksgiving...","[(hysterical, JJ), (thanks, NNS), (laugh, IN),..."
4,Elon Musk can show us the way but it’s we who ...,011add077f4d2da3,1595483162289016832,2022-11-23,18:23:23,"[elon, musk, show, us, way, it’s, hard, work, ...",[],"[elon, musk, show, us, way, it, hard, work, ma...","[elon, musk, show, u, way, it’s, hard, work, m...","[(elon, NN), (musk, NN), (show, NN), (u, JJ), ..."
...,...,...,...,...,...,...,...,...,...,...
690,@jljacobson Her opinion of him has changed dra...,01a9a39529b27f36,1595853104565686273,2022-11-24,18:53:24,"[opinion, changed, dramatically, done, listen,...",[],"[opinion, chang, dramat, done, listen, latest,...","[opinion, changed, dramatically, done, listen,...","[(opinion, NN), (changed, VBD), (dramatically,..."
691,@RonFilipkowski @elonmusk Another right wing i...,7142eb97ae21e839,1595853147792187397,2022-11-24,18:53:34,"[another, right, wing, pleading, lunacy, reins...",[],"[anoth, right, wing, plead, lunaci, reinstat, ...","[another, right, wing, pleading, lunacy, reins...","[(another, DT), (right, NN), (wing, VBG), (ple..."
692,@MattWallace888 Another mindless shit post…it’...,9dafd05b1158873b,1595853341531471872,2022-11-24,18:54:20,"[another, mindless, shit, post…it’s, private, ...",[],"[anoth, mindless, shit, post…it, privat, entit...","[another, mindless, shit, post…it’s, private, ...","[(another, DT), (mindless, NN), (shit, VBD), (..."
693,@jbblawmd Well I burned the skin on the oven w...,dd9c503d6c35364b,1595853364713213957,2022-11-24,18:54:26,"[well, burned, skin, oven, cooking, turkey, le...",[],"[well, burn, skin, oven, cook, turkey, least, ...","[well, burned, skin, oven, cooking, turkey, le...","[(well, RB), (burned, VBN), (skin, NN), (oven,..."


---
### Positive/Negative

In [15]:
pos_words = []
f = open('positive-words.txt')
for line in f:
    try:
        pos_words.append(line.replace('\n', ''))
    except:
        pass

f.close()
len(pos_words)

2006

In [16]:
neg_words = []
f = open('negative-words.txt', encoding='ISO-8859-1')
for line in f:
    try:
        neg_words.append(line.replace('\n', ''))
    except:
        pass

f.close()
len(neg_words)

4783

In [17]:
df['Count'] = None
df['Pos_count'] = None
df['Neg_count'] = None

for x in range(len(df)):
    df['Count'][x] = len(df['Words'][x])
    df['Pos_count'][x] = len([word for word in df['Lemmed'][x] if word in pos_words])
    df['Neg_count'][x] = len([word for word in df['Lemmed'][x] if word in neg_words])
    

In [18]:
# Remove rows with no words.
df_fix = df.copy()
for idx, row in df[df['Count'] == 0].iterrows():
    df_fix = df_fix.drop(idx)

In [19]:
df_fix['Positive_ratio'] = df_fix['Pos_count']/df_fix['Count']
df_fix['Negative_ratio'] = df_fix['Neg_count']/df_fix['Count']

In [20]:
df_fix

,Text,Place_id,Tweet_id,Date,Time,Words,Hashtags,Stemmed,Lemmed,POS_Tags,Count,Pos_count,Neg_count,Positive_ratio,Negative_ratio
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,6079430106fe2a39,1595482958764425218,2022-11-23,18:22:34,"[econusyougovamerica, approve, disapprove, elo...",[],"[econusyougovamerica, approv, disapprov, elon,...","[econusyougovamerica, approve, disapprove, elo...","[(econusyougovamerica, RB), (approve, VB), (di...",37,2,1,0.054054,0.027027
1,Twitter and Elon has the same vibes when Trump...,94965b2c45386f87,1595483029048369152,2022-11-23,18:22:51,"[twitter, elon, vibes, trump, got, elected, la...",[],"[twitter, elon, vibe, trump, got, elect, laugh...","[twitter, elon, vibe, trump, got, elected, lau...","[(twitter, NN), (elon, NN), (vibe, NN), (trump...",19,1,1,0.052632,0.052632
2,@RonFilipkowski Then why do I see him so frequ...,2d83c71ce16cd187,1595483099735154688,2022-11-23,18:23:08,"[see, frequently, twitter, stop, posting]",[],"[see, frequent, twitter, stop, post]","[see, frequently, twitter, stop, posting]","[(see, VB), (frequently, RB), (twitter, JJR), ...",5,0,0,0.0,0.0
3,@timothy_wall @elonmusk This is hysterical. Th...,018244f41aa15ac3,1595483141879353344,2022-11-23,18:23:18,"[hysterical, thanks, laugh, prep, thanksgiving...",[],"[hyster, thank, laugh, prep, thanksgiv, need]","[hysterical, thanks, laugh, prep, thanksgiving...","[(hysterical, JJ), (thanks, NNS), (laugh, IN),...",6,0,1,0.0,0.166667
4,Elon Musk can show us the way but it’s we who ...,011add077f4d2da3,1595483162289016832,2022-11-23,18:23:23,"[elon, musk, show, us, way, it’s, hard, work, ...",[],"[elon, musk, show, us, way, it, hard, work, ma...","[elon, musk, show, u, way, it’s, hard, work, m...","[(elon, NN), (musk, NN), (show, NN), (u, JJ), ...",10,1,1,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,@jljacobson Her opinion of him has changed dra...,01a9a39529b27f36,1595853104565686273,2022-11-24,18:53:24,"[opinion, changed, dramatically, done, listen,...",[],"[opinion, chang, dramat, done, listen, latest,...","[opinion, changed, dramatically, done, listen,...","[(opinion, NN), (changed, VBD), (dramatically,...",7,0,0,0.0,0.0
691,@RonFilipkowski @elonmusk Another right wing i...,7142eb97ae21e839,1595853147792187397,2022-11-24,18:53:34,"[another, right, wing, pleading, lunacy, reins...",[],"[anoth, right, wing, plead, lunaci, reinstat, ...","[another, right, wing, pleading, lunacy, reins...","[(another, DT), (right, NN), (wing, VBG), (ple...",16,1,0,0.0625,0.0
692,@MattWallace888 Another mindless shit post…it’...,9dafd05b1158873b,1595853341531471872,2022-11-24,18:54:20,"[another, mindless, shit, post…it’s, private, ...",[],"[anoth, mindless, shit, post…it, privat, entit...","[another, mindless, shit, post…it’s, private, ...","[(another, DT), (mindless, NN), (shit, VBD), (...",22,1,4,0.045455,0.181818
693,@jbblawmd Well I burned the skin on the oven w...,dd9c503d6c35364b,1595853364713213957,2022-11-24,18:54:26,"[well, burned, skin, oven, cooking, turkey, le...",[],"[well, burn, skin, oven, cook, turkey, least, ...","[well, burned, skin, oven, cooking, turkey, le...","[(well, RB), (burned, VBN), (skin, NN), (oven,...",11,2,1,0.181818,0.090909


---
#### Sort By Positive Ratio

In [25]:
df_fix.sort_values('Positive_ratio')[::-1][:50]

,Text,Place_id,Tweet_id,Date,Time,Words,Hashtags,Stemmed,Lemmed,POS_Tags,Count,Pos_count,Neg_count,Positive_ratio,Negative_ratio
336,@Sinfulskin2 Trump,b463d3bd6064861b,1595813253455683586,2022-11-24,16:15:03,[trump],[],[trump],[trump],"[(trump, NN)]",1,1,0,1.0,0.0
156,@mazemoore Love it!! 😂😂😂,1c69a67ad480e1b1,1595497183511515149,2022-11-23,19:19:06,[love],[],[love],[love],"[(love, NN)]",1,1,0,1.0,0.0
367,@Davidlaz Interesting! 🤔🙂,3b77caf94bfc81fe,1595817277047574529,2022-11-24,16:31:02,[interesting],[],[interest],[interesting],"[(interesting, VBG)]",1,1,0,1.0,0.0
153,@AlCappuccinoIT You have more patience than I,3b77caf94bfc81fe,1595496936110686208,2022-11-23,19:18:07,[patience],[],[patienc],[patience],"[(patience, NN)]",1,1,0,1.0,0.0
557,@bennyjohnson Outstanding!,2cce4a9792d2d92e,1595835514615386112,2022-11-24,17:43:30,[outstanding],[],[outstand],[outstanding],"[(outstanding, JJ)]",1,1,0,1.0,0.0
184,@realTuckFrumper Good,013885130d700578,1595500171353522176,2022-11-23,19:30:58,[good],[],[good],[good],"[(good, JJ)]",1,1,0,1.0,0.0
14,@AFCtrent_ @elonmusk You are so right!,57ea83a184f3142d,1595484299603722262,2022-11-23,18:27:54,[right],[],[right],[right],"[(right, NN)]",1,1,0,1.0,0.0
242,@elonmusk Am I doing it right? https://t.co/zZ...,c3f37afa9efcf94b,1595505528238931968,2022-11-23,19:52:15,[right],[],[right],[right],"[(right, NN)]",1,1,0,1.0,0.0
462,@RBReich Like this during trump's inauguration...,00c23e64064f65f5,1595825124225290247,2022-11-24,17:02:13,"[like, trumps, inauguration]",[],"[like, trump, inaugur]","[like, trump, inauguration]","[(like, IN), (trump, NN), (inauguration, NN)]",3,2,0,0.666667,0.0
608,@LauraFlowD @WholeMarsBlog I’m ready for some ...,c7ef5f3368b68777,1595842277305368579,2022-11-24,18:10:22,"[i’m, ready, goodness]",[],"[i'm, readi, good]","[i’m, ready, goodness]","[(i’m, JJ), (ready, JJ), (goodness, NN)]",3,2,0,0.666667,0.0


---
#### One Word Responses

In [24]:
len(df_fix[df_fix['Count'] == 1])

45